# test

## include scripts

### include notebook core

In [ ]:
. ../../../../scripts/nbs_header.ps1

### Include core functions script

In [ ]:
. ../../../../scripts/core.ps1

### Include spiral library

In [ ]:
. ../../../../lib/spiral/lib.ps1

## execute project commands

### run notebook with retries using spiral supervisor

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --execute-command "../../../../workspace/target/release/spiral_builder$(_exe) dib --path test.dib --retries 3" } | Invoke-Block

00:00:00 verbose #1 networking.test_port_open / { port = 13806; ex = System.AggregateException: One or more errors occurred. (Connection refused) }
00:00:00   debug #1 runtime.execute_with_options_async / { options = { command = ../../../../workspace/target/release/spiral_builder dib --path test.dib --retries 3; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = None; stdin = None; trace = true; working_directory = None } }
00:00:00 verbose #2 > 00:00:00   debug #1 spiral_builder.main / { args = Array(MutCell(["dib", "--path", "test.dib", "--retries", "3"])) }
00:00:00 verbose #3 > 00:00:00   debug #2 runtime.execute_with_options / { file_name = dotnet; arguments = ["repl", "--exit-after-run", "--run", "/home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.dib", "--output-path", "/home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.dib.ipynb"]; options = { command = dotnet repl --exit-after-run --run "/home/runner/wo

### parse the .dib file into .spi format with dibparser

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(_exe) test.dib spi } | Invoke-Block

00:00:00   debug #1 writeDibCode / output: Spi / path: test.dib
00:00:00   debug #2 parseDibCode / output: Spi / file: test.dib


### build .fsx file from .spi using supervisor

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --build-file test.spi test.fsx } | Invoke-Block

00:00:00 verbose #1 networking.test_port_open / { port = 13806; ex = System.AggregateException: One or more errors occurred. (Connection refused) }
00:00:00 verbose #2 networking.test_port_open / { port = 13806; ex = System.AggregateException: One or more errors occurred. (Connection refused) }
00:00:00   debug #1 Supervisor.buildFile / takeWhileInclusive / outputContent:
 / errors: [] / typeErrorCount: 0 / retry: 0 / path: test.spi
00:00:00   debug #2 Supervisor.buildFile / AsyncSeq.scan / outputContent:
 / errors: [] / outputContentResult:  / typeErrorCount: 0 / retry: 0 / error:  / path: test.spi
00:00:00   debug #3 Supervisor.buildFile / takeWhileInclusive / outputContent:
 / errors: [] / typeErrorCount: 0 / retry: 0 / path: test.spi
00:00:00 verbose #4 Supervisor.sendJson / port: 13805 / json: {"FileOpen":{"spiText":"/// # test (Polyglot)\nnominal i = ()\nnominal e = ()\nnominal s = ()\nnomin...t\u003E\u0022\n","uri":"file:///home/runner/work/polyglot/polyglot/apps/spiral/temp/tes

## compile and format the project

### compile project with fable targeting optimized rust

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.19.3: F# to Rust compiler (status: alpha)

Thanks to the contributor! @dgchurchill
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
.> dotnet restore test.fable-temp.csproj -p:FABLE_COMPILER=true -p:FABLE_COMPILER_4=true -p:FABLE_COMPILER_RUST=true
  Determining projects to restore...
  Paket version 8.1.0-alpha004+7aa412f49b32de979c3d5acde07e88e6d47c965b
  The last full restore is still up to date. Nothing left to do.
  Total time taken: 0 milliseconds
  Paket version 8.1.0-alpha004+7aa412f49b32de979c3d5acde07e88e6d47c965b
  Restoring /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fable-temp.csproj
  Starting restore process.
  Total time taken: 0 milliseconds
  Restored /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fable-temp.csproj (in 311 ms).
.> dotnet restore /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fsproj
  Determining projects to restore...
  Restored /home/runner/work/polyglot/polyglot/a

### fix formatting issues in the .rs file using regex and set-content

In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
    | FixRust `
| Set-Content test.rs

### format the rust code using cargo fmt

In [ ]:
cargo fmt --

## build and test the project

### build the project in release mode using nightly rust compiler

In [ ]:
cargo +nightly build --release

   Compiling linux-raw-sys v0.4.14
   Compiling rand_core v0.6.4
   Compiling num-traits v0.2.19
   Compiling syn v2.0.70
   Compiling rand_chacha v0.3.1
   Compiling libm v0.2.8
   Compiling rustix v0.38.34
   Compiling wait-timeout v0.2.0
   Compiling quick-error v1.2.3
   Compiling bit-vec v0.6.3
   Compiling bit-set v0.5.3
   Compiling rand v0.8.5
   Compiling rand_xorshift v0.3.0
   Compiling memchr v2.7.4
   Compiling unarray v0.1.4
   Compiling lazy_static v1.5.0
   Compiling fable_library_rust v0.1.0 (/home/runner/work/polyglot/polyglot/lib/rust/fable/fable_modules/fable-library-rust)
   Compiling tempfile v3.10.1
   Compiling nom v7.1.3
   Compiling thiserror-impl v1.0.61
   Compiling rusty-fork v0.3.0
   Compiling proptest v1.4.0
   Compiling thiserror v1.0.61
   Compiling spiral_temp_test v0.0.1 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test)
  --> /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/./main.rs:16:6
   |
16 | enum Item {
   |      ^^^^
   |
 

### run release tests with output enabled

In [ ]:
{ cargo +nightly test --release -- --show-output } | Invoke-Block

   Compiling bitflags v2.6.0
   Compiling linux-raw-sys v0.4.14
   Compiling fastrand v2.1.0
   Compiling wait-timeout v0.2.0
   Compiling bit-vec v0.6.3
   Compiling fnv v1.0.7
   Compiling quick-error v1.2.3
   Compiling num-traits v0.2.19
   Compiling bit-set v0.5.3
   Compiling rand v0.8.5
   Compiling rustix v0.38.34
   Compiling rand_xorshift v0.3.0
   Compiling minimal-lexical v0.2.1
   Compiling unarray v0.1.4
   Compiling memchr v2.7.4
   Compiling fable_library_rust v0.1.0 (/home/runner/work/polyglot/polyglot/lib/rust/fable/fable_modules/fable-library-rust)
   Compiling thiserror v1.0.61
   Compiling nom v7.1.3
   Compiling tempfile v3.10.1
   Compiling rusty-fork v0.3.0
   Compiling proptest v1.4.0
   Compiling spiral_temp_test v0.0.1 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test)
    Finished `release` profile [optimized] target(s) in 15.47s
     Running unittests main.rs (/home/runner/work/polyglot/polyglot/workspace/target/release/deps/spiral_temp_test-9729c7

### execute the binary in release mode

In [ ]:
{ . $ScriptDir/../../../../workspace/target/release/spiral_temp_test$(_exe) } | Invoke-Block

app=test
